# PROJECT : REAL-TIME AQI ANALYSIS AND VISUALIZATION OF INDIA

Importing necessary libraries:

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap

- STEP 1 -> Accessing data using API

In [3]:
# Details of API at:- https://aqicn.org/api/
base_url = "https://api.waqi.info"

# Storing a special User token
tok = "6cec9fff29506692c24dcb2a603e5561461da99d" # Token available at :- https://aqicn.org/data-platform/token/#/

# Storing latitude and longitude in format: (lat, long)-> Bottom Left ; (lat, lon)-> Top Right
latlngbox = "8,61,37,97" # Location of India is 8N 61E to 37N, 97E approx
trail_url=f"/map/bounds/?latlng={latlngbox}&token={tok}"

# Calling data from URL
my_data = pd.read_json(base_url + trail_url)
my_data

,status,data
0,ok,"{'lat': 25.892357, 'lon': 86.590325, 'uid': 13..."
1,ok,"{'lat': 26.2271665, 'lon': 84.3570427, 'uid': ..."
2,ok,"{'lat': 32.5, 'lon': 80.1161, 'uid': 7069, 'aq..."
3,ok,"{'lat': 29.4723508, 'lon': 77.7194031, 'uid': ..."
4,ok,"{'lat': 30.719859, 'lon': 76.738637, 'uid': 13..."
...,...,...
198,ok,"{'lat': 13.428828, 'lon': 77.731418, 'uid': 11..."
199,ok,"{'lat': 24.952822, 'lon': 84.002396, 'uid': 13..."
200,ok,"{'lat': 25.44919916, 'lon': 81.82735986, 'uid'..."
201,ok,"{'lat': 29.14056, 'lon': 75.744941, 'uid': 118..."


- STEP 2 -> Converting Json data to a readable DataFrame

In [4]:
all_rows = []
for each_row in my_data['data']:
    all_rows.append([each_row['station']['name'],each_row['lat'],each_row['lon'],each_row['aqi']])
df = pd.DataFrame(all_rows, columns=['station_name', 'lat', 'lon', 'aqi'])
df

,station_name,lat,lon,aqi
0,"Police Line, Saharsa, India",25.892357,86.590325,468
1,"Chitragupta Nagar, Siwan, India",26.227166,84.357043,423
2,"Ngari Station, Ngari Sanai (阿里地区阿里监测站)",32.500000,80.116100,42
3,"New Mandi, Muzaffarnagar, India",29.472351,77.719403,229
4,"Sector-53, Chandigarh, India",30.719859,76.738637,409
...,...,...,...,...
198,"Chikkaballapur Rural, Chikkaballapur, India",13.428828,77.731418,160
199,"Dada Peer, Sasaram, India",24.952822,84.002396,403
200,"Nagar Nigam, Prayagraj, India",25.449199,81.827360,157
201,"Urban Estate-II, Hisar, India",29.140560,75.744941,178


- STEP 3 -> Cleaning the DataFrame

In [49]:
# Converting Invalid parse in AQI to NaN
df['aqi'] = pd.to_numeric(df.aqi, errors='coerce')
# Removing NaN Values in AQIs Column
df1 = df.dropna(subset = ['aqi'])

- STEP 4 -> Visualisation

# Visualization Of Live_HeatMap using folium ->

In [114]:
df2 = df1[['lat', 'lon', 'aqi']] # Second Data Frame without State Name
init_loc = [22, 80] # Approx location info of Centre of India to set as centre of map

m = folium.Map(location = init_loc, zoom_start = 5)
heat_aqi = HeatMap(df2, radius = 15, blur = 10)
m.add_child(heat_aqi)
m # Show the map

# Visualization Of Various Stations on Map using folium ->

In [113]:
m2 = folium.Map(location = init_loc, zoom_start= 5)
for _, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    station = row['station_name'] + ' AQI=' + str(row['aqi'])
    station_aqi = row['aqi']
    if station_aqi > 300: ## Black for very bad AQI
        pop_color = 'black'
    elif station_aqi > 200:
        pop_color = 'darkred' ## Red for moderate AQI
    else:
        pop_color = 'cadetblue' ## Blue for good AQI
    folium.Marker(location= [lat, lon],popup = station,icon = folium.Icon(color = pop_color)).add_to(m2)
m2 # Display map